In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
온라인 = pd.read_csv('/Users/yaelinjo/GitHub/Recosys_Project/온라인_전처리_final_32columns.csv')
온라인.head()

,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


In [4]:
온라인.columns[3:-2]

Index(['hit_seq', 'hit_pss_tm', 'time_length', 'holiday_diff', 'keyword',
       'action_type_0', 'action_type_1', 'action_type_2', 'action_type_3',
       'action_type_4', 'biz_unit_A01', 'biz_unit_A02', 'biz_unit_A03',
       'sess_dt_201907', 'sess_dt_201908', 'sess_dt_201909', 'trfc_src_DIRECT',
       'trfc_src_PORTAL_1', 'trfc_src_PORTAL_2', 'trfc_src_PORTAL_3',
       'trfc_src_PUSH', 'trfc_src_WEBSITE', 'trfc_src_unknown',
       'dvc_ctg_nm_PC', 'dvc_ctg_nm_mobile_app', 'dvc_ctg_nm_mobile_web',
       'dvc_ctg_nm_unknown'],
      dtype='object')

In [5]:
len(온라인.columns[3:-2])

27

# <br>
# 종속변수 생성
현재 세션의 구매 여부

In [6]:
# 현재 세션의 구매를 예측하기 위한 종속변수 생성
구매여부 = 온라인[['clnt_id', 'sess_id', 'buy']].groupby(['clnt_id', 'sess_id']).sum()
구매여부.buy = 구매여부.buy.apply(lambda x:0 if x == 0 else 1)
구매여부 = 구매여부.sort_index()
구매여부 = 구매여부.reset_index()

In [7]:
구매여부['unique_id'] = list(map(lambda x,y: str(x)+'_'+str(y), 구매여부.clnt_id, 구매여부.sess_id))

In [8]:
구매여부

,clnt_id,sess_id,buy,unique_id
0,1,1,0,1_1
1,1,2,0,1_2
2,2,1,1,2_1
3,2,2,0,2_2
4,2,3,0,2_3
...,...,...,...,...
367144,72428,2,0,72428_2
367145,72428,3,0,72428_3
367146,72428,4,0,72428_4
367147,72428,5,0,72428_5


# <br>
# 구매 이전의 클릭로그 대상

In [9]:
idx1 = 온라인.unique_id.drop_duplicates().index.tolist()
idx2 = idx1[1:] + [len(온라인)]
idx3 = []
for i, j in tqdm_notebook(zip(idx1, idx2), total=len(idx1)):
    temp = 온라인.buy.iloc[i:j]
    try:
        idx3.append(temp[temp == 1].index[0])
    except:
        idx3.append(j)

In [28]:
온라인_구매이전_idx = []

for i, j in tqdm_notebook(zip(idx1, idx3), total=len(idx1)):
    온라인_구매이전_idx.extend(온라인.iloc[i:j].index.tolist())

In [31]:
온라인2 = 온라인.iloc[온라인_구매이전_idx].copy()

In [32]:
온라인2.head()

,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


# <br>
# flatten

In [34]:
def to_flat(df):
    cc = df.groupby(['unique_id']).cumcount() + 1
    flat_df = df.set_index(['unique_id', cc]).unstack().sort_index(1, level=1)
    flat_df.columns = ['_'.join(map(str,i)) for i in flat_df.columns]
    flat_df.reset_index(inplace=True)
    return flat_df

In [35]:
온라인2.drop(['clnt_id', 'sess_id', 'trans_id', 'buy'], axis=1, inplace=True)

In [36]:
온라인_flat = to_flat(온라인2)

In [37]:
온라인_flat = 온라인_flat.merge(구매여부, left_on='unique_id', right_on='unique_id')
온라인_flat.sort_values(by=['clnt_id','sess_id'], inplace=True)
온라인_flat.reset_index(drop=True, inplace=True)

In [38]:
# max 357 * 27 features + 4 = 9643
온라인_flat.shape

(359779, 9643)

In [39]:
# 패딩처리 시에도 0으로 패딩했으니까
# NaN 값들은 0으로 대체
온라인_flat.fillna(0, inplace=True)

In [40]:
온라인_flat.head()

,unique_id,action_type_0_1,action_type_1_1,action_type_2_1,action_type_3_1,action_type_4_1,biz_unit_A01_1,biz_unit_A02_1,biz_unit_A03_1,dvc_ctg_nm_PC_1,...,trfc_src_DIRECT_357,trfc_src_PORTAL_1_357,trfc_src_PORTAL_2_357,trfc_src_PORTAL_3_357,trfc_src_PUSH_357,trfc_src_WEBSITE_357,trfc_src_unknown_357,clnt_id,sess_id,buy
0,1_1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0
1,1_2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,0
2,2_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1,1
3,2_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0
4,2_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,3,0


# <br>
# [2-2]현재 세션(1개)의 첫번째 구매 이전 클릭 로그 대상

In [19]:
from keras.preprocessing import sequence
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0

In [20]:
features = 온라인_flat.columns[1:-3]

In [21]:
온라인_x = 온라인_flat.iloc[:, 1:-3]
온라인_x = np.array(온라인_x)

In [22]:
# session 당 구매 여부
온라인_y = 온라인_flat.buy

In [23]:
def make_padding_and_oversample(X, Y, length=10):
    smote = SMOTE(random_state=0)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)
    return X_resampled, Y_resampled

In [24]:
X_resampled, Y_resampled = make_padding_and_oversample(온라인_x, 온라인_y, length=10)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, Y_resampled, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

(81713, 270)
(35021, 270)


# <br>
# 모델링

In [26]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import xgboost as xgb

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.layers import Dropout

import pickle
from joblib import dump, load

In [27]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [28]:
scores = dict()

### Gaussian Naive Bayes

In [29]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.57
F1 score: 0.65
Precision: 0.55
Recall: 0.79


In [30]:
scores['Gaussian Naive Bayes'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                                  precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [31]:
# dump(clf, '1-3-Gaussian.joblib')

### Decision Tree

In [32]:
clf2 = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.71
F1 score: 0.71
Precision: 0.71
Recall: 0.71


In [33]:
scores['Decision Tree'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred),
                           precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [34]:
feature_check = {}
for name, importance in zip(features, clf2.feature_importances_):
    feature_check[name] = importance

In [35]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
action_type_3_2,0.138492
action_type_0_1,0.060195
time_length_10,0.036032
action_type_2_10,0.034688
time_length_5,0.030759
...,...
trfc_src_PORTAL_3_9,0.000000
hit_seq_7,0.000000
biz_unit_A02_2,0.000000
hit_seq_1,0.000000


In [36]:
# dump(clf2, '1-3-DecisionTree.joblib')

### XGBOOST

In [37]:
clf3 = xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=3, verbosity=2, random_state=0).fit(X_train, y_train)
y_pred = clf3.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

[15:29:01] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[15:29:01] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[15:29:02] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15:29:02] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15:29:03] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15:29:03] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[15:

In [38]:
scores['XGboost'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [39]:
feature_check = {}
for name, importance in zip(features, clf3.feature_importances_):
    feature_check[name] = importance

In [40]:
pd.DataFrame.from_dict(feature_check, orient='index', columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)

,feature_importance
dvc_ctg_nm_unknown_2,0.135423
trfc_src_DIRECT_7,0.116404
sess_dt_201908_2,0.076774
sess_dt_201909_5,0.062521
dvc_ctg_nm_unknown_4,0.058914
...,...
biz_unit_A02_6,0.000000
biz_unit_A03_6,0.000000
dvc_ctg_nm_PC_6,0.000000
dvc_ctg_nm_mobile_app_6,0.000000


In [41]:
# dump(clf3, '1-3-xgb.joblib')

### Logistic Regression

In [42]:
clf4 = LogisticRegression(max_iter=5000, random_state=0).fit(X_train, y_train)
y_pred = clf4.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.64
F1 score: 0.65
Precision: 0.63
Recall: 0.68


In [43]:
scores['Logistic Regression'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [44]:
# dump(clf4, '1-3-LogisticRegression.joblib')

### Linear SVM

In [46]:
clf5 = LinearSVC(random_state=0).fit(X_train, y_train)
y_pred = clf5.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('F1 score: %.2f' % f1_score(y_test, y_pred))
print('Precision: %.2f' % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))

Accuracy: 0.54
F1 score: 0.41
Precision: 0.56
Recall: 0.33


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [47]:
scores['Linear SVM'] = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred)]

In [48]:
# dump(clf4, '1-3-LinearSVM.joblib')

### MLP(DNN)

In [49]:
def models(train):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(train.shape[1],)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr= 0.001, rho = 0.9), metrics=['acc', f1_m, precision_m, recall_m])
    return model

In [50]:
model = models(X_train)
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                8672      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 9,217
Trainable params: 9,217
Non-trainable params: 0
_________________________________________________________________


In [51]:
history = model.fit(X_train, y_train, epochs=25, batch_size=1000, validation_data=(X_test, y_test), verbose=2, shuffle=True)


Train on 81713 samples, validate on 35021 samples
Epoch 1/25
 - 1s - loss: 4.2283 - acc: 0.5039 - f1_m: 0.4161 - precision_m: 0.5027 - recall_m: 0.4995 - val_loss: 1.9828 - val_acc: 0.5173 - val_f1_m: 0.6538 - val_precision_m: 0.5059 - val_recall_m: 0.9253
Epoch 2/25
 - 1s - loss: 2.3428 - acc: 0.5108 - f1_m: 0.3831 - precision_m: 0.5056 - recall_m: 0.5112 - val_loss: 2.5690 - val_acc: 0.5111 - val_f1_m: 0.6639 - val_precision_m: 0.5023 - val_recall_m: 0.9801
Epoch 3/25
 - 1s - loss: 2.1022 - acc: 0.5132 - f1_m: 0.3693 - precision_m: 0.5089 - recall_m: 0.5028 - val_loss: 1.5521 - val_acc: 0.5372 - val_f1_m: 0.6677 - val_precision_m: 0.5181 - val_recall_m: 0.9399
Epoch 4/25
 - 1s - loss: 1.9501 - acc: 0.5133 - f1_m: 0.3742 - precision_m: 0.5123 - recall_m: 0.5067 - val_loss: 2.2602 - val_acc: 0.5176 - val_f1_m: 0.6663 - val_precision_m: 0.5063 - val_recall_m: 0.9753
Epoch 5/25
 - 1s - loss: 1.8319 - acc: 0.5181 - f1_m: 0.3859 - precision_m: 0.5174 - recall_m: 0.5103 - val_loss: 1.6928 

In [52]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)

35021/35021 [==============================] - 2s 45us/step


In [53]:
scores['DNN'] = [accuracy, f1_score, precision, recall]

In [54]:
print(loss)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

0.6718268989018931
0.5913594961166382
0.5524645447731018
0.5999837517738342
0.5274563431739807


In [55]:
# model.save('1-3-DNN.h5')

In [56]:
pd.DataFrame.from_dict(scores, orient='index', columns=['Accuracy', 'F1-Score', 'Precision', 'Recall'])

,Accuracy,F1-Score,Precision,Recall
Gaussian Naive Bayes,0.571800,0.648014,0.547213,0.794338
Decision Tree,0.711487,0.710088,0.708138,0.712050
XGboost,0.780389,0.735350,0.914577,0.614858
Logistic Regression,0.643357,0.654915,0.629889,0.682012
Linear SVM,0.537106,0.413389,0.556888,0.328691
DNN,0.591359,0.552465,0.599984,0.527456
